## ここでは，データベース作成時に利用した独特なpandasの挙動を示す

### あるカラムに同じ要素を持つ行をグループ化 

以下のようなDataFrameがあったとする．

In [1]:
import pandas as pd

In [4]:
df = pd.DataFrame([["Shinjuku", "Tokyo"],
                   ["Kawasaki", "Kanagawa"],
                   ["Urawa", "Saitama"],
                   ["Shibuya", "Tokyo"],
                   ["Yokohama", "Kanagawa"],
                  ],columns=["city", "pref"])
df

,city,pref
0,Shinjuku,Tokyo
1,Kawasaki,Kanagawa
2,Urawa,Saitama
3,Shibuya,Tokyo
4,Yokohama,Kanagawa


これを，prefカラムの値ごとに，cityのリストを取得したいとする．つまり{"pref":"city"のリスト}となるようにしたいとする

In [13]:
grouped = df.groupby("pref")
grouped

このgroupbyは何らかの関数を適用しないと実体を持たない(例えば平均など)

In [15]:
grouped_df = pd.DataFrame(grouped)
grouped_df

,0,1
0,Kanagawa,city pref 1 Kawasaki Kanagawa 4 ...
1,Saitama,city pref 2 Urawa Saitama
2,Tokyo,city pref 0 Shinjuku Tokyo 3 Shib...


上はDataFrameのコンストラクタに渡した場合．これじゃ意味が分からない．そこでリスト化してみる．

In [17]:
list(grouped)

[('Kanagawa',
         city      pref
  1  Kawasaki  Kanagawa
  4  Yokohama  Kanagawa),
 ('Saitama',
      city     pref
  2  Urawa  Saitama),
 ('Tokyo',
         city   pref
  0  Shinjuku  Tokyo
  3   Shibuya  Tokyo)]

どうやらそれぞれがDataFrameのリストになるようである．

In [19]:
list(grouped["city"])

[('Kanagawa',
  1    Kawasaki
  4    Yokohama
  Name: city, dtype: object),
 ('Saitama',
  2    Urawa
  Name: city, dtype: object),
 ('Tokyo',
  0    Shinjuku
  3     Shibuya
  Name: city, dtype: object)]

上のようにカラム名を指定すると，それぞれのDataFrameのカラムを指定できる．普通のDataFrameのようにlocを使わないことに注意

In [23]:
dict_group = dict(list(grouped["city"]))
dict_group

{'Kanagawa': 1    Kawasaki
 4    Yokohama
 Name: city, dtype: object,
 'Saitama': 2    Urawa
 Name: city, dtype: object,
 'Tokyo': 0    Shinjuku
 3     Shibuya
 Name: city, dtype: object}

上の例はかなりそれっぽいが，それぞれが変なインデックス(元のインデックス)を含むDataFrameになってしまう．そこで，それぞれのグループに関してvaluesを取得する関数を利用(apply)してみよう．

In [32]:
def get_values(df):
    return df.values

group_array = grouped["city"].apply(get_values)
group_array

pref
Kanagawa    [Kawasaki, Yokohama]
Saitama                  [Urawa]
Tokyo        [Shinjuku, Shibuya]
Name: city, dtype: object

今回はpd.Seriesが返ってくる．それはインデックスがprefになった各要素にndarrayを持つSeriesである．もしapplyにmean, max, min等を与えれば，各要素がその関数を適用された値になるわけである．

In [33]:
group_array.loc["Kanagawa"]

array(['Kawasaki', 'Yokohama'], dtype=object)

そこで，このSeriesをdictにしてあげればよい．

In [36]:
dict(group_array)

{'Kanagawa': array(['Kawasaki', 'Yokohama'], dtype=object),
 'Saitama': array(['Urawa'], dtype=object),
 'Tokyo': array(['Shinjuku', 'Shibuya'], dtype=object)}

これはpd.Seriesを辞書にすることでインデックスをキーとし，その要素を値とする仕様による

In [37]:
series = pd.Series([1,2,3,4,5],index=["A","B","C","D","E"])
dict(series)

{'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5}

### まとめ 

In [38]:
def get_values(df):
    return df.values

group_array = grouped["city"].apply(get_values)
dict_group_array = dict(group_array)
dict_group_array

{'Kanagawa': array(['Kawasaki', 'Yokohama'], dtype=object),
 'Saitama': array(['Urawa'], dtype=object),
 'Tokyo': array(['Shinjuku', 'Shibuya'], dtype=object)}

### リサンプル用のaggのための辞書作り 

df.resample.aggを利用する際，{"カラム名":関数名や対応文字列}とする辞書が必要になるこれをfor文を用いずに作成する

In [55]:
df = pd.DataFrame([[1,1,1,1,1,1,1,1,1,1]],columns=["Open_0","High_0","Low_0","Close_0","Volume_0","Open_1","High_1","Low_1","Close_1","Volume_1"])
df.columns

Index(['Open_0', 'High_0', 'Low_0', 'Close_0', 'Volume_0', 'Open_1', 'High_1',
       'Low_1', 'Close_1', 'Volume_1'],
      dtype='object')

これをOpen_等ごとに代入する値を変えたいとする．そのようにカラム名の文字列パターンを扱うには，pandas.index.strの各種メソッドを利用する．ある文字列から始まる場合はstart_withを利用する．

In [51]:
df.columns.str.startswith("Open")

array([ True, False, False, False, False,  True, False, False, False,
       False])

このブール値を利用してカラム名を取得できる．

In [52]:
df.loc[:,df.columns.str.startswith("Open")]

,Open_0,Open_1
0,1,1


そこで，カラム名をインデックスとするSeriesを作成し，そのSeriesの要素の値にそれぞれ関数あるいは対応文字列を与え，最後にそのSeriesを辞書に変更する

In [58]:
column_series = pd.Series(len(df.columns)*[None], index=df.columns)
column_series.loc[column_series.index.str.startswith("Open")] = "first"
column_series.loc[column_series.index.str.startswith("High")] = "max"
column_series.loc[column_series.index.str.startswith("Low")] = "min"
column_series.loc[column_series.index.str.startswith("Close")] = "last"
column_series.loc[column_series.index.str.startswith("Volume")] = "sum"
dict(column_series)

{'Open_0': 'first',
 'High_0': 'max',
 'Low_0': 'min',
 'Close_0': 'last',
 'Volume_0': 'sum',
 'Open_1': 'first',
 'High_1': 'max',
 'Low_1': 'min',
 'Close_1': 'last',
 'Volume_1': 'sum'}

正規表現を利用した場合

In [60]:
import re

In [61]:
column_series = pd.Series(len(df.columns)*[None], index=df.columns)
column_series.loc[column_series.index.str.match(re.compile("Open_.+"))] = "first"
column_series.loc[column_series.index.str.match("High_.+")] = "max"
column_series.loc[column_series.index.str.match("Low_.+")] = "min"
column_series.loc[column_series.index.str.match("Close_.+")] = "last"
column_series.loc[column_series.index.str.match("Volume_.+")] = "sum"
dict(column_series)

{'Open_0': 'first',
 'High_0': 'max',
 'Low_0': 'min',
 'Close_0': 'last',
 'Volume_0': 'sum',
 'Open_1': 'first',
 'High_1': 'max',
 'Low_1': 'min',
 'Close_1': 'last',
 'Volume_1': 'sum'}